# Using the linalg module of NumPy

### Import NumPy
Bootstrapping the term structure requires us to invert and multiple matrices and arrays-common operations of linear algebra.  The NumPy module <font color='green'>linalg</font> includes functions that manipulate matrices and arrays. This notebook uses the module to demonstrate the bootstrapping applications detailed in *Inferring zero prices from the present value function*. The initial step is to import the NumPy library. The standard <font color='green'>try</font> and <font color='green'>except</font> is used to import the library.  



In [1]:
# Need to import NumPy
try:
    import numpy as np

# install if necessary
except:
    !pip -q install numpy
    import numpy as np

### Check the rank of <font color='green'>payoffs_two_bonds</font> to determine if it is invertible
The payoff matrices in, *Inferring zero prices from the present value function*, include data columns for both six months and one year. Inspection reveals that both columns contribute unique information—a condition that may not hold when this approach is expanded to include multiple bonds.

A critical issue arises if a date's column offers no incremental information: the payoff matrix, or the product of its transpose and the payoff matrix, are non-invertible or singular and a solution cannot be calculated. Although obvious in our two-date examples, a general procedure is needed to test for unique information in each column.

The rank of a matrix is defined by the number of columns that cannot be reproduced by a linear combination of the other columns. For our two-date examples, we need two independent columns or a rank of two. The <font color='green'>linalg matrix_rank</font> function returns the number of independent columns.





 ### Demonstrating the <font color='green'>matrix_rank</font> function.
 The next code cell creates a three column matrix, but the third column equals the first column plus the second column. Column three adds no new information.  The rank of the matrix is two.

*  The <font color='green'> matrix_a</font>. is created.

*  The linalg function <font color='green'>matrix_rank</font> returns two as the rank of <font color='green'> matrix_a</font>.

*  The number of rows and columns of <font color='green'> matrix_a</font> is returned with the <font color='green'> shape</font> attribute.

*  Invertibility is determined by testing the equality of <font color='green'> num_columns_a</font> and <font color='green'> rank_a</font>

In [2]:
# Create the NumPy two-dimensional array
matrix_a = np.array([[1, 2, 3],
                      [4, 5, 9],
                      [3, 5, 8]])

# Determine the rank with matrix_rank function
rank_a = np.linalg.matrix_rank(matrix_a)

# Compare number of columns to rank
# shape attribute returns a tuple (number of rows,number of columns)
num_row_a,num_columns_a=matrix_a.shape

# Test for invertibility
invertible=num_columns_a==rank_a
print(f"Is matrix_a invertible?: {invertible}--'Number of columns: {num_columns_a}\
---Rank of a: {rank_a}")

Is matrix_a invertible?: False--'Number of columns: 3---Rank of a: 2


## <font color='green'>Application: Find the rank of non-square two-dimensional array</font>


<div style=
    border-left: 12px solid green;
    font-family: 'Garamond', serif;
    font-size: 17px;
    line-height: 1.5;
    padding: 15px">
<br>
Calculate the rank of the array:

$\begin{pmatrix}
1&10&5  \\
2 &102&25  \\
\end{pmatrix}$

 For hints, see [Chapter Four Hints: Non-square two-dimensional array](https://colab.research.google.com/drive/1hSV2ArS-kEm3soW-sbIBTjWnnD3nMvkh#scrollTo=bjK2ippQxMBu),  and check the [expected results here](https://colab.research.google.com/drive/1hrv5WQPxxuto7jIy1I6_fdyeAyhNCLaY#scrollTo=bjK2ippQxMBu).

<br>
</div>


## The function <font color='green'>check_rank</font> tests if a NumPy array has full column rank.

*   $\text{rank of matrix} \leq \min(\text{number of rows},\text{number of columns})$
*   $\text{rank of rows} = \text{rank of columns}$


*   ##### Input must be a NumPy array
*   ##### <font color='green'>shape</font> attribute returns (number of rows,number of columns).
*   ##### <font color='green'>matrix_rank</font> function returns number of indpendent columns.
*   ##### function returns <font color='green'>independent_columns</font>-the truth value of equality of rank and number of columns.
<br>

If the number of rows exceeds the number of columns, the rows of a matrix may be linearly dependent.  An important result when bootstrapping the term structure with more bonds (rows) than dates (columns).




In [3]:
def check_rank(array):
  '''
  Checks if a matrix has Full Column Rank (all columns are linearly independent).

    Args:
        array (np.ndarray): The matrix to check.

    Returns:
        bool: True if all columns are independent, False if there is redundancy.
  '''
  import numpy as np
  # Validate input
  if not isinstance(array,np.ndarray):
    raise TypeError("Input must be a NumPy array.")

  # Get number of columns and rows
  # Full column rank implies that number of rows >= number of columns
  array_rows,array_columns=array.shape
  if array_rows<array_columns:
    return False

  # Calculate rank
  array_rank=np.linalg.matrix_rank(array)


  # If is_full_rank  true, least-squares or exact solutions possible
  is_full_rank=array_columns==array_rank
  return is_full_rank

## Two equations and two unknowns: the exact solution.
The first example of *Inferring zero prices from the present value function* has two dates and two bonds.  An exact solution is possible.


1.   The payoff matrix <font color='green'>payoffs_two_bonds</font> and the bond price vector <font color='green'>prices_two_bonds</font> are defined.
2.   The full column rank of <font color='green'>payoffs_two_bonds</font> is verified with the <font color='green'>check_rank</font> function.



### Define the payoff matrix. prices, and full rank of payoff matrix.

In [4]:
# Define the payoff matrix (two-dimensional array) for the two bond example
payoffs_two_bonds=np.array([[100,0],  #six month bond
                           [2,102]])  #one-year coupon bond

# Define the price array for the two bond example
prices_two_bonds=np.array([97.5,      #six month bond
                           100])      #one-year coupon bond

#Verify the rank of the payoff matrix
full_rank=check_rank(payoffs_two_bonds)
print(f'Payoff matrix is full rank?  {full_rank}')

Payoff matrix is full rank?  True


### Get the inverse of the <font color='green'>payoffs_two_bond</font> array with the linalg module.

*   The function <font color='green'>inv</font> of linalg returns the inverse
*   The inverse is verified by multiplying the inverse and the original array with the NumPy <font color='green'>dot</font> function.



In [5]:
# Use inv function of linalg module to calculate inverse and display result
payoffs_two_bond_inverse=np.linalg.inv(payoffs_two_bonds)
print('Inverse of payoffs_two_bonds \n',payoffs_two_bond_inverse)

# Use dot function to multiply the inverse and original matrix and display results
# The product of a matrix and its inverse equals an identity
# matrix (one's on the main diagonal and zeroes elsewhere).
should_be_identity=np.dot(payoffs_two_bond_inverse,payoffs_two_bonds)
print('Inverse multiplied into payoffs_two_bond: Main diagonal 1 all other zero \n',should_be_identity)

Inverse of payoffs_two_bonds 
 [[ 0.01        0.        ]
 [-0.00019608  0.00980392]]
Inverse multiplied into payoffs_two_bond: Main diagonal 1 all other zero 
 [[1. 0.]
 [0. 1.]]


### Calculate the present value factors or zero prices


*   Multiply the payoff inverse and bond price vector to get the present value factors (zero prices) using the <font color='green'>dot</font> function.
*   Multiply the payoff matrix and present value factors to get the bond prices.



In [6]:
# Calculate and display two bond present value factors

# Multiple inverse of payoff and bond prices
present_value_factors_two_bonds=np.dot(payoffs_two_bond_inverse,prices_two_bonds)
# Display present value factors
print('Present Value Factors',present_value_factors_two_bonds)

# Confirm present value factors by confirming bond prices
# Multiply payoffs into present value factors
calculated_two_bond_prices=np.dot(payoffs_two_bonds,present_value_factors_two_bonds)
# Display results
print('Payoffs Times Present Value Factors',calculated_two_bond_prices)

Present Value Factors [0.975      0.96127451]
Payoffs Times Present Value Factors [ 97.5 100. ]


### The linalg function <font color='green'>solve</font>



*   Solving for the present value factors with the functions <font color='green'>inv</font> and <font color='green'>dot</font> requires two lines of code
    * <font color='green'>payoffs_two_bond_inverse=np.linalg.inv(payoffs_two_bonds)
    * present_value_factors_two_bonds=np.dot(payoffs_two_bond_inverse,prices_two_bonds)</font>
*   Solving for present value factors with linalg function <font color='green'>solve</font> takes one line of code.
     * <font color='green'>present_value_factors_two_bonds=np.linalg.solve(payoffs_two_bonds,prices_two_bonds)</font>




In [7]:
present_value_factors_two_bonds=np.linalg.solve(payoffs_two_bonds,prices_two_bonds)
present_value_factors_two_bonds

array([0.975     , 0.96127451])

## The three bond least-squares solution
The second example of *Inferring zero prices from the present value function* has two dates and three bonds.  An exact solution is not possible; however, the payoff matrix has full column rank and the present value factors are solved with the least-squares method.



1.   The payoff matrix <font color='green'>payoffs_three_bonds</font> and the bond price vector <font color='green'>prices_three_bonds</font> are defined.
2.   The full column rank of <font color='green'>payoffs_three_bonds</font> is verified with the <font color='green'>check_rank</font> function.


### Define the payoff matrix. prices, and full rank of payoff matrix.

In [8]:
#the payoff and bond price matrixes are defined
payoffs_three_bonds=np.array([[100,0],    # six-month bond
                             [2,102],     # one-year coupon bond
                              [0,100]])   # one-year zero-coupon bond

prices_three_bonds=np.array([97.5,          # six-month bond
                           100,           #one-year coupon bond
                           96])           #one-year zero-coupon bond

#Verify the rank of the payoff matrix with check_rank function
full_rank=check_rank(payoffs_three_bonds)
print(f'Payoff matrix is full rank?  {full_rank}')

Payoff matrix is full rank?  True


### Calculate the transpose of <font color='green'>payoffs_three_bonds</font> with transpose attribute of NumPy arrays.
The easiest way to get the transpose of a NumPy array is to get its transpose attribute with .T

In [9]:
# Assign the transpose attribute .T to payoffs_three_bonds
payoffs_three_bonds_transpose=payoffs_three_bonds.T

# Display the original and transposed arrays
print('The original array \n',payoffs_three_bonds)
print('The transposed array \n',payoffs_three_bonds_transpose)

The original array 
 [[100   0]
 [  2 102]
 [  0 100]]
The transposed array 
 [[100   2   0]
 [  0 102 100]]


### Calculate and display multiplying transpose into original matrix and into price matrix
*   Multiplication of the transpose of <font color='green'>payoffs_three_bonds</font> into the original payoff matrix. Using the <font color='green'>dot</font> functions.
*   Multiplication of transpose of payoff matrix into the price array, <font color='green'>prices_three_bonds</font>.

In [10]:
# Using dot function to multiply transpose into original payoff matrix
three_payoff_transpose_original=np.dot(payoffs_three_bonds_transpose,payoffs_three_bonds)

# display results
print('Multiplying Transpose of payoff matrix into payoff matrix \n',three_payoff_transpose_original)

# Use dot function to multiply transpose into prices_three_bonds
three_payoff_transpose_prices=np.dot(payoffs_three_bonds_transpose,prices_three_bonds)
# display result
print('Multiplying Transpose of payoff matrix into three boond price array \n',three_payoff_transpose_prices)

Multiplying Transpose of payoff matrix into payoff matrix 
 [[10004   204]
 [  204 20404]]
Multiplying Transpose of payoff matrix into three boond price array 
 [ 9950. 19800.]


### Solve for present value factors


*   Use <font color='green'>inv</font> function to invert the product of the transpose and the original payoff matrix.
*   Multiply the inverse into the product of the transpose of the payoff matrix and the three bond price array.
*   Display the least-square estimates of the present value factors.




In [11]:
inverse_transpose_original=np.linalg.inv(three_payoff_transpose_original)
least_square_pv_factors=np.dot(inverse_transpose_original,three_payoff_transpose_prices)
print('Least-Squares PV Three Bonds \n',least_square_pv_factors)

Least-Squares PV Three Bonds 
 [0.97501274 0.96064975]


The estimated present value factors differ slightly from the factors derived using the payoffs and prices of two bonds. The small differences suggests that the third bond adds little to the information set.

### The linalg function <font color='green'>lstsq</font>

The <font color='green'>lstsq</font> function requires a payoff matrix and price array. The function returns the estimated present value factors, the sum of the squared differences between the bond prices and the prices forecasted by the estimated factors, the column rank of the payoff matrix, and a measure of the information content of each column of the payoff matrix is close to having less than full rank.  Our purpose is to estimate the present value factors.

*   Solving for the present value factors with the functions <font color='green'>inv</font> and <font color='green'>dot</font> requires five lines of code
    * <font color='green'>payoffs_three_bonds_transpose=payoffs_three_bonds.T
    * three_payoff_transpose_original=np.dot(payoffs_three_bonds_transpose,payoffs_three_bonds)
    * three_payoff_transpose_prices=np.dot(payoffs_three_bonds_transpose,prices_three_bonds)
    *  inverse_transpose_original=np.linalg.inv(three_payoff_transpose_original)
    * least_square_pv_factors=np.dot(inverse_transpose_original,three_payoff_transpose_prices)</font>
*   Solving for present value factors with linalg function <font color='green'>lstsq</font> takes one line of code.
     * <font color='green'>least_square_pv_factors=np.linalg.lstsq(payoffs_three_bonds,prices_three_bonds)</font>




### The <font color='green'>lstsq</font> function returns four values:



1.   The estimated present value factors
2.   The sum of the squared differences between the actual bond prices and those computed with the estimated present value factors.
3.   The rank of the payoff matrix.
4.   A measure of how much information content of each column. The ratio of the largest to the smallest indicate the information content of the columns of the matrix.

Our concern is the estimated present value factors.


In [12]:
least_square_pv_factors=np.linalg.lstsq(payoffs_three_bonds,prices_three_bonds)
results=(f'Six-month present value factor--{least_square_pv_factors[0][0]: .4f}\
   One-year present value factor---{least_square_pv_factors[0][1]: .4f} \n'
f'Sum of squared differences---{least_square_pv_factors[1][0]: .4f}\n'
f'Column rank of payoff matrix---{least_square_pv_factors[2]}\n'
f'Information content of each column--{least_square_pv_factors[3]}')
print(results)

Six-month present value factor-- 0.9750   One-year present value factor--- 0.9606 
Sum of squared differences--- 0.0083
Column rank of payoff matrix---2
Information content of each column--[142.85657143 100.        ]


/tmp/ipykernel_38232/1878391595.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  least_square_pv_factors=np.linalg.lstsq(payoffs_three_bonds,prices_three_bonds)
